Sustainability team project BDM

In [24]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from datetime import datetime, date
from pyspark.sql import Row
from py4j.java_gateway import JavaGateway
import pandas as pd
import os 
import sys 

os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable

In [25]:
# Creazione contesto
gateway = JavaGateway()
conf = SparkConf().setAppName("test1.pal").setMaster("local")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=test1.pal, master=local) created by __init__ at C:\Users\palaz\AppData\Local\Temp\ipykernel_8312\428759442.py:4 

In [ ]:
df = pd.read_csv(r"C:\Users\palaz\OneDrive\Desktop\University\UNIPA 2.0\I ANNO\II semestre\Data Processing\BDM - Big Data Management\Progetto 2\BigDataProject\data\sondaggio_sostenibilità_unipa.csv")

#Eliminazione colonne non necessarie: colonne informazioni generali + colonna 81 
df.drop(columns=df.columns[:16] , inplace=True)
df.drop(columns=df.columns[-1], inplace=True)

df.head()

,Quante volte alla settimana fai la spesa?,Quanti supermercati visiti quando fai la spesa?,"Per una vita più sostenibile, una buona abitudine per gli acquisti è quello di servirsi di prodotti locali e/o di propria produzione. In quale delle seguenti condizioni ti rispecchi maggiormente?","Per incentivare la spesa sostenibile, sono stati creati dei servizi online per l’acquisto e spedizione a casa di tutti i prodotti biologici, siti come BioDiscount, Greenweez, La Boutique del Biologico. Sfrutti tali servizi?","All’interno dei supermercati, presti attenzione ai prodotti con etichetta ecologica (basso consumo CO2, allevamento a terra)?",Sacchetti della spesa,Prodotti per la casa e per indumenti,Acqua potabile,Molte associazioni coinvolgono i supermercati per la raccolta di cibo all’uscita dei supermercati. Quanto spesso dai in beneficenza cibaria?,"In media, quante volte al mese acquisti vestiti?",...,Rifiuti - Lavoro o Università [C'è un'attenzione nel ridurre l'uso della plasica],Rifiuti - Lavoro o Università [C'è un'attenzione alla gestione di materiali organici],Rifiuti - Lavoro o Università [C'è un'attenzione alla gestione di materiali inorganici],"Rifiuti - Casa [Vengono implementate le 3R: Riduzione, Riuso e Riciclo]",Rifiuti - Casa [C'è un'attenzione nel ridurre l'uso della plasica],Rifiuti - Casa [C'è un'attenzione alla gestione di materiali organici],Rifiuti - Casa [C'è un'attenzione alla gestione di materiali inorganici],"Inserire il numero di eventi (conferenze, workshop, sensibilizzazione, formazione pratica, festival, etc.) legati ai problemi di sostenibilità e dell'ambiente organizzati dalla propria azienda e/o università (media per anno degli ultimi 3 anni). \r\nSelezionare una delle seguenti opzioni:","Inserire il numero totale di attività organizzate dalle organizzazioni studentesche o da eventuali organizzazioni interne al proprio ambiente di lavoro per anno. Ad esempio, seminari, webinars, formazione, eventi sportivi, bazaar di materiali riciclati, comunità di divulgazione, etc.. \r\nSelezionare una delle seguenti opzioni:","La presenza di strutture ""green"" all'interno del proprio campus e/o della propria azienda che siano accessibili al pubblico, per esempio durante attività culturali, indicano un impatto più ampio di tali strutture nei loro dintorni (Festival Culturali, teatro, performance musicali, esibizioni, attività virtuali, etc.).\r\nSelezionare una delle seguenti opzioni:"
0,≥ 5,≥ 5,Acquisto spesso (media di 2 volte al mese) ai ...,"No, li conosco ma non li uso","No, non guardo le etichette",Uso i sacchetti del supermercato e li butto via,"Riacquisto i prodotti, non vi sono servizi all...",Riacquisto le casse di acqua,1 volta all'anno,3,...,1-50%,1-50%,In fase di Preparazione,In fase di Preparazione,1-50%,In fase di Preparazione,1-50%,0,1 - 2,2 eventi per anno
1,2,2,Mi servo esclusivamente dei supermercati,"No, li conosco ma non li uso","Si, raramente acquisto prodotti con etichetta ...","Non uso i sacchetti del supermercato, uso sacc...","Riacquisto i prodotti, non vi sono servizi all...","Riacquisto le casse di acqua, non sono present...",1 volta all'anno,2,...,1-50%,1-50%,51-75%,In Progetto,In Progetto,1-50%,In Progetto,1 - 4,3 - 4,1 evento per anno
2,2,2,Acquisto molti (media di più di 2 volte al mes...,"No, non ne ero a conoscenza","No, non guardo le etichette",Uso i sacchetti del supermercato e li riutilizzo,Riacquisto i prodotti,Riacquisto le casse di acqua,2 volte all'anno,2,...,76-100%,76-100%,76-100%,76-100%,76-100%,76-100%,76-100%,1 - 4,1 - 2,3 eventi per anno
3,1,2,Acquisto spesso (media di 2 volte al mese) ai ...,"No, non ne ero a conoscenza","Si, raramente acquisto prodotti con etichetta ...","Non uso i sacchetti del supermercato, uso sacc...",Riacquisto i prodotti,Riacquisto le casse di acqua,0,1,...,1-50%,1-50%,1-50%,1-50%,1-50%,1-50%,1-50%,0,0,Nessuna
4,2,1,Mi servo esclusivamente dei supermercati,"No, non ne ero a conoscenza","Si, raramente acquisto prodotti con etichetta ...","No

In [ ]:
#Creazione Dataframe tramite spark con nuovo df
df_0 = spark.createDataFrame(df)
df_0.show()


+-----------------------------------------+-----------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------+---------------------+------------------------------------+--------------------+--------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------+---------------------------------------------------------------------------------------------